# Workspace for Project Euler problems

## Problem 9

A Pythagorean triplet is a set of three natural numbers, $a < b < c$, for which,

$$a^2 + b^2 = c^2$$

For example, $3^2 + 4^2 = 9 + 16 = 25 = 5^2$.

There exists exactly one Pythagorean triplet for which $a + b + c = 1000$.
Find the product $abc$.

In [20]:
import primetools as pt
import numbers, math
import tools.misc as misc

### Some random substitution just looking for stuff

1.1: $a + b + c = 1000$

1.2: $c = 1000 - a - b$

2: $a^2 + b^2 = c^2$

3, 1.1 squared:
$a^2 + 2ab + b^2 + 2bc + c^2 + 2ac = 1000000$

4, Substitute 2: $2a^2 + 2ab + 2b^2 + c(2b + 2a) = 1000000$

5, Substitute 1.2: $2a^2 + 2ab + 2b^2 + (1000 - a - b)(2b + 2a) = 1000000$

6: $(1000 - a - b)(2b + 2a) = 2000b + 2000a - 4ab - 2a^2 - 2b^2$

7: $-4ab - 2a^2 - 2b^2 = -2(2ab + a^2 + b^2) = -2(a + b)^2$

8, Substitute 6: $-2ab + 2000a + 2000b = 1000000$

### Ok what did we learn

From [8], we know that $-2ab$ must be a multiple of 1000, since the other terms in the equation are, so $ab$ must be a multiple of 500 between them.